# If use model with right hand

In [7]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="SymbolDatabase.GetPrototype() is deprecated")

import pickle
import cv2
import mediapipe as mp
import numpy as np
import tkinter as tk
import threading
import time
import logging
import translatation
import speech

# globals
selected_language = "zul_Latn"  # Default language is isiZulu

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Load the model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Initialize the video capture
cap = cv2.VideoCapture(0)

# Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.7, max_num_hands=1)

# Labels dictionary
labels_dict = {
    0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f',
    6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l',
    12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r',
    18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x',
    24: 'y', 25: 'z', 26: '0', 27: '1', 28: '2',
    29: '3', 30: '4', 31: '5', 32: '6', 33: '7', 34: '8', 35: '9',
    36: 'I love You', 37: 'yes', 38: 'No', 39: 'Hello', 40: 'Thanks',
    41: 'Sorry', 42: 'space'
}

# Create a tkinter window
root = tk.Tk()
root.title("ASL Prediction")

# Set window size and center it
window_width = 500
window_height = 400
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
position_top = int(screen_height / 2 - window_height / 2)
position_right = int(screen_width / 2 - window_width / 2)
root.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")

# Function to update selected language when a button is pressed
def set_language(lang_code):
    global selected_language
    selected_language = lang_code

# Create frames for organization
language_frame = tk.Frame(root)
language_frame.pack(pady=10)

translation_frame = tk.Frame(root)
translation_frame.pack(pady=20)

# Function to exit the application and stop both Tkinter and OpenCV
def exit_app():
    global cap
    logging.info('Exiting application...')
    if cap.isOpened():
        cap.release()  # Release the video capture
    cv2.destroyAllWindows()  # Close OpenCV windows
    root.quit()  # Stop the Tkinter main loop
    root.destroy()  # Close the Tkinter window


# Create language buttons
# Create language buttons in row 0 (aligned to the left)
afrikaans_button = tk.Button(language_frame, text="Afrikaans", command=lambda: set_language("afr_Latn"), font=("Helvetica", 12))
afrikaans_button.grid(row=0, column=0, padx=10)

isizulu_button = tk.Button(language_frame, text="isiZulu", command=lambda: set_language("zul_Latn"), font=("Helvetica", 12))
isizulu_button.grid(row=0, column=1, padx=10)

setswana_button = tk.Button(language_frame, text="Setswana", command=lambda: set_language("tsn_Latn"), font=("Helvetica", 12))
setswana_button.grid(row=0, column=2, padx=10)

sesotho_button = tk.Button(language_frame, text="Sesotho", command=lambda: set_language("sot_Latn"), font=("Helvetica", 12))
sesotho_button.grid(row=0, column=3, padx=10)


# Create language buttons in row 1 (centered)
isiSwati_button = tk.Button(language_frame, text="isiSwati", command=lambda: set_language("ssw_Latn"), font=("Helvetica", 12))
isiSwati_button.grid(row=1, column=1, padx=10)

xitsonga_button = tk.Button(language_frame, text="Xitsonga", command=lambda: set_language("tso_Latn"), font=("Helvetica", 12))
xitsonga_button.grid(row=1, column=2, padx=10)

isiXhosa_button = tk.Button(language_frame, text="isiXhosa", command=lambda: set_language("xho_Latn"), font=("Helvetica", 12))
isiXhosa_button.grid(row=1, column=0, padx=10)

sepedi_button = tk.Button(language_frame, text="Sepedi", command=lambda: set_language("nso_Latn"), font=("Helvetica", 12))
sepedi_button.grid(row=1, column=3, padx=10)



# Create a label to display the translated text
translated_label = tk.Label(translation_frame, text="", font=("Helvetica", 14), wraplength=400)
translated_label.pack()

# Create a text field in tkinter
text_field = tk.Text(translation_frame, height=2, width=40, font=("Helvetica", 16))
text_field.pack(pady=10)

# Create a button to trigger translation
def trigger_translation():
    input_text = text_field.get("1.0", tk.END).strip()  # Get the text from the text field
    translated_text = translatation.translate_from_box(input_text, selected_language)  # Get the translation
    lang_code_into_plain=speech.convert_lang_code(selected_language)
    # Display the translated text in the label
    translated_label.config(text=f" '{input_text}' \n translated into {lang_code_into_plain} is \n'{translated_text}' ")  # Update the label with the translation
    speech.synthesize_and_play(translated_text, selected_language)  # Synthesize and play the translated text

translate_button = tk.Button(translation_frame, text="Translate", command=trigger_translation, font=("Helvetica", 12))
translate_button.pack(pady=5)

# Create a clear text button
def clear_text():
    text_field.delete('1.0', tk.END)  # Clear the text field
    logging.info('Text cleared.')

clear_button = tk.Button(translation_frame, text="Clear Text", command=clear_text, font=("Helvetica", 12))
clear_button.pack(pady=5)

# Create an "Exit" button
exit_button = tk.Button(root, text="Exit", command=exit_app, font=("Helvetica", 12), bg="red", fg="white")
exit_button.pack(pady=20)


# Variable to store the previous prediction and time
prev_prediction = None
word_count = 0  # Track how many words have been written

# Variables to track the detected character and delay counter
last_detected_character = None
fixed_character = ""
delayCounter = 0
start_time = time.time()

# Function to update the tkinter text field by appending the new predicted character
def update_text_field(text):
    if text == 'space':
        text_field.insert(tk.END, ' ')  # Append a space
    else:
        text_field.insert(tk.END, text + '')  # Append new character
    logging.info(f'Word added: {text if text != "space" else "space (represented as space)"}')

# Function to run video capture and ASL prediction in a separate thread
def run():
    global last_detected_character, fixed_character, delayCounter, start_time

    # Make the OpenCV window resizable
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 800, 600)  # Optionally set the initial window size

    while True:
        data_aux = []
        x_ = []
        y_ = []

        ret, frame = cap.read()

        if not ret:
            break

        H, W, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))
                
                margin = 50
                x1 = int(min(x_) * W) - margin
                y1 = int(min(y_) * H) - margin
                x2 = int(max(x_) * W) + margin
                y2 = int(max(y_) * H) + margin

                # Make prediction using the model
                prediction = model.predict([np.asarray(data_aux)])
                predicted_character = labels_dict[int(prediction[0])]

                # Draw a rectangle and the predicted character on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                            cv2.LINE_AA)

                current_time = time.time()

                # Timer logic: Check if the predicted character is the same for more than 1 second
                if predicted_character == last_detected_character:
                    if (current_time - start_time) >= 1.0:  # Class fixed after 1 second
                        fixed_character = predicted_character
                        if delayCounter == 0:  # Add character once after it stabilizes for 1 second
                            update_text_field(fixed_character)
                            delayCounter = 1
                else:
                    # Reset the timer when a new character is detected
                    start_time = current_time
                    last_detected_character = predicted_character
                    delayCounter = 0  # Reset delay counter for a new character

        # Show the video feed with the prediction
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Start the video capture in a separate thread to keep tkinter responsive
threading.Thread(target=run, daemon=True).start()

# Start the tkinter main loop
root.mainloop()




c:\Users\dYlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\dYlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dYlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase